In [7]:
from pymongo import MongoClient
import pandas as pd
import time
import geopandas as gdp
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
from getpass import getpass


In [8]:
client = MongoClient("localhost:27017")

db = client["ironhack"]


In [9]:
db.list_collection_names()

['nyrest', 'Crunchbase', 'restaurants', 'nyneigh']

In [10]:
c = db.get_collection("Crunchbase")

In [12]:
token = getpass()

In [13]:
company_category = set()

In [ ]:
# category1 = { "category_code" : {"$exists": true, "$ne": "null"}}
category_regex = {"category_code": {"$regex": "(?i)^(?:u[ix] develop|frontend|backend|gam(?:ing|er)|tech)$"}}
description_regex = {"description": {"$regex": "(?i)^(?:u[ix] develop|frontend|backend|gam(?:ing|er)|tech)$"}}
tags_regex = {"tag_list": {"$regex": "(?i)^(?:u[ix] develop|frontend|backend|gam(?:ing|er)|tech)$"}}

projection_name_category_tags = {"_id"=0, "name"=1, "offices.0.country_code"=1, "offices.0.city"=1, "offices.0.zip_code"=1, "offices.0.latitude"=1, "offices.0.longitude"=1}

list(c.find)